In [ ]:
# The purpose of this code is to develop a GIF with a map of Minnesota (MN) successively adding MN state highways-just for fun

In [143]:
import pandas as pd
import geopandas as gpd
import descartes
import matplotlib.pyplot as plt
import numpy as np
import shapely.geometry
import imageio # to make the GIF file
import os

In [5]:
roads = 'STREETS_LOAD.shp'

In [308]:
states = '/Users/matthewwilliamhaas/Documents/shapefile_data_sources/cb_2018_us_state_20m/cb_2018_us_state_20m.shp'
map_states = gpd.read_file(states)

In [309]:
map_states[map_states.STATEFP == '27']

,STATEFP,STATENS,AFFGEOID,GEOID,STUSPS,NAME,LSAD,ALAND,AWATER,geometry
28,27,00662849,0400000US27,27,MN,Minnesota,00,206228939448,18945217189,"POLYGON ((-97.22904 49.00069, -96.93096 48.999..."


In [310]:
MN = map_states[(map_states.STATEFP == '27')]
MN = MN.to_crs({'init' : 'epsg:26915'})

In [6]:
# read in data
road_data = gpd.read_file(roads)

In [164]:
road_data[(MN280)]

,STREET_NAM,STREET_PRE,BASE_NAME,STREET_TYP,STREET_SUF,E_911,ROUTE_SYST,ROUTE_NUMB,TIS_CODE,DIVIDED_ON,TRAFFIC_DI,SHAPE_Leng,geometry
6306,MNTH 280,None,None,None,None,D,03,280,0300000280,M,I,565.695996,"LINESTRING (483915.876 4981782.748, 483866.013..."
6403,MNTH 280,None,None,None,None,D,03,280,0300000280,M,D,764.654286,"LINESTRING (483750.340 4982975.841, 483746.245..."
37938,MNTH 280,None,None,None,None,D,03,280,0300000280,M,D,189.294034,"LINESTRING (484100.094 4984125.000, 484043.563..."
40741,MNTH 280,None,None,None,None,D,03,280,0300000280,M,D,1026.468621,"LINESTRING (483940.625 4984029.888, 483909.872..."
40857,MNTH 280,None,None,None,None,D,03,280,0300000280,M,I,440.110534,"LINESTRING (483898.473 4978496.745, 483945.937..."
41691,MNTH 280,None,None,None,None,D,03,280,0300000280,M,I,193.678763,"LINESTRING (483954.751 4981171.896, 483955.062..."
104919,MNTH 280,None,None,None,None,D,03,280,0300000280,M,D,373.467493,"LINESTRING (483774.303 4982213.099, 483784.394..."
107875,MNTH 280,None,None,None,None,D,03,280,0300000280,M,D,150.525192,"LINESTRING (484003.531 4980385.999, 484005.281..."
115869,MNTH 280,None,None,None,None,D,03,280,0300000280,M,D,816.532840,"LINESTRING (483930.719 4981197.500, 483930.844..."
166658,MNTH 280,None,None,None,None,D,03,280,0300000280,M,D,67.053405,"LINESTRING (483760.190 4983042.158, 483750.678..."


In [69]:
# used this code for the map of MN and WI--not sure if it will be useful here or not
#TH61 = TH61.to_crs({"init" : "epsg:26915"})

In [330]:
# Convert code for making one-off highway maps to a function to do in a high-throughput manner.
# Needed arguments: 1) path to image (highway sign), 2) indicator of current highway to plot geodata, and 3) output filename

# Initiate empty list to hold highways that have already been plotted
#alreadyPlotted = [] # Keep outside function definitation so it doesn't get cleared every time the function is called
#geoDataAll = gpd.GeoSeries(crs='epsg:26915') # make empty geopandas geoseries dataframe

def MakeHwyMap (imagefile, currentHwy, outfile):
    global geoDataAll
    data = road_data.STREET_NAM == currentHwy
    geoData = road_data[(data)].geometry
    im = plt.imread(imagefile)
    fig, ax = plt.subplots(1, figsize = (20,15))
    MN.plot(color = 'white', linewidth = 1.0, ax = ax, edgecolor = 'black')
    plt.title('Minnesota state ("trunk") highway system', fontsize = 60)
    #if currentHwy in alreadyPlotted:
    geoDataAll.plot(color ='grey', linewidth = 3.0, ax = ax, edgecolor = 'black')
    #else:
    geoData.plot(color ='red', linewidth = 3.0, ax = ax, edgecolor = 'black')
    #alreadyPlotted.append(currentHwy)
    geoDataAll = geoDataAll.append(geoData)
    newax = fig.add_axes([0.8, 0.5, 0.2, 0.3], anchor = 'NE', zorder = 1)
    newax.imshow(im)
    ax.axis('off') # turn off the axis
    newax.axis('off')
    plt.savefig(outfile)
    plt.close('all')

In [331]:
infile = open('minnesota_highway_list.csv', 'r', encoding='utf-8-sig') # open file
highways = infile.readlines()  # begin reading data

out_PNG_list = [] # make empty list to hold names of output files to make GIF at end
#alreadyPlotted = []
geoDataAll = gpd.GeoSeries(crs='epsg:26915') # make empty geopandas geoseries dataframe

# Run the MakeHwyMap function for each line in the file (each line represents a highway)
for line in highways:
    line = line.rstrip()
    line = line.split(',')
    print(line[0])
    MakeHwyMap(imagefile = line[0], currentHwy = line[1], outfile = line[2])
    out_PNG_list.append(line[2])

/Users/matthewwilliamhaas/Documents/MN_highway_signs/MN-1.png


/Users/matthewwilliamhaas/opt/anaconda3/envs/map/lib/python3.7/site-packages/geopandas/plotting.py:309: UserWarning: The GeoSeries you are attempting to plot is empty. Nothing has been displayed.
  UserWarning,


/Users/matthewwilliamhaas/Documents/MN_highway_signs/MN-3.png
/Users/matthewwilliamhaas/Documents/MN_highway_signs/MN-4.png
/Users/matthewwilliamhaas/Documents/MN_highway_signs/MN-5.png
/Users/matthewwilliamhaas/Documents/MN_highway_signs/MN-6.png
/Users/matthewwilliamhaas/Documents/MN_highway_signs/MN-7.png
/Users/matthewwilliamhaas/Documents/MN_highway_signs/MN-9.png
/Users/matthewwilliamhaas/Documents/MN_highway_signs/MN-11.png
/Users/matthewwilliamhaas/Documents/MN_highway_signs/MN-13.png
/Users/matthewwilliamhaas/Documents/MN_highway_signs/MN-15.png
/Users/matthewwilliamhaas/Documents/MN_highway_signs/MN-16.png
/Users/matthewwilliamhaas/Documents/MN_highway_signs/MN-18.png
/Users/matthewwilliamhaas/Documents/MN_highway_signs/MN-19.png
/Users/matthewwilliamhaas/Documents/MN_highway_signs/MN-20.png
/Users/matthewwilliamhaas/Documents/MN_highway_signs/MN-21.png
/Users/matthewwilliamhaas/Documents/MN_highway_signs/MN-22.png
/Users/matthewwilliamhaas/Documents/MN_highway_signs/MN-23.pn

/Users/matthewwilliamhaas/opt/anaconda3/envs/map/lib/python3.7/site-packages/geopandas/plotting.py:309: UserWarning: The GeoSeries you are attempting to plot is empty. Nothing has been displayed.
  UserWarning,


/Users/matthewwilliamhaas/Documents/MN_highway_signs/MN-40.png
/Users/matthewwilliamhaas/Documents/MN_highway_signs/MN-41.png
/Users/matthewwilliamhaas/Documents/MN_highway_signs/MN-42.png
/Users/matthewwilliamhaas/Documents/MN_highway_signs/MN-43.png
/Users/matthewwilliamhaas/Documents/MN_highway_signs/MN-44.png
/Users/matthewwilliamhaas/Documents/MN_highway_signs/MN-45.png
/Users/matthewwilliamhaas/Documents/MN_highway_signs/MN-46.png
/Users/matthewwilliamhaas/Documents/MN_highway_signs/MN-47.png
/Users/matthewwilliamhaas/Documents/MN_highway_signs/MN-48.png
/Users/matthewwilliamhaas/Documents/MN_highway_signs/MN-50.png


/Users/matthewwilliamhaas/opt/anaconda3/envs/map/lib/python3.7/site-packages/geopandas/plotting.py:309: UserWarning: The GeoSeries you are attempting to plot is empty. Nothing has been displayed.
  UserWarning,


/Users/matthewwilliamhaas/Documents/MN_highway_signs/MN-51.png
/Users/matthewwilliamhaas/Documents/MN_highway_signs/MN-54.png
/Users/matthewwilliamhaas/Documents/MN_highway_signs/MN-55.png
/Users/matthewwilliamhaas/Documents/MN_highway_signs/MN-56.png
/Users/matthewwilliamhaas/Documents/MN_highway_signs/MN-57.png
/Users/matthewwilliamhaas/Documents/MN_highway_signs/MN-58.png
/Users/matthewwilliamhaas/Documents/MN_highway_signs/MN-60.png


ValueError: invalid PNG header

In [332]:
# make the GIF file
with imageio.get_writer('movie.gif', mode = 'I', duration = 1) as writer:
    for filename in out_PNG_list:
        image = imageio.imread(filename)
        writer.append_data(image)

In [287]:
road_data[(road_data.STREET_NAM == 'Snelling Ave')]

,STREET_NAM,STREET_PRE,BASE_NAME,STREET_TYP,STREET_SUF,E_911,ROUTE_SYST,ROUTE_NUMB,TIS_CODE,DIVIDED_ON,TRAFFIC_DI,SHAPE_Leng,geometry
17886,Snelling Ave,None,Snelling,Ave,None,U,10,8,1035350008,T,B,101.528228,"LINESTRING (486772.812 4995511.000, 486770.376..."
111765,Snelling Ave,None,Snelling,Ave,None,U,10,626,1010400626,T,B,13.135221,"LINESTRING (569827.562 5184774.001, 569826.473..."
113760,Snelling Ave,None,Snelling,Ave,None,U,10,8,1035350008,T,B,100.022963,"LINESTRING (486770.376 4995612.499, 486768.280..."
133996,Snelling Ave,None,Snelling,Ave,None,U,10,8,1035350008,T,B,100.503536,"LINESTRING (486773.655 4995410.500, 486772.812..."
172414,Snelling Ave,None,Snelling,Ave,None,U,10,8,1035350008,T,B,98.562199,"LINESTRING (486777.156 4995312.000, 486773.655..."
210609,Snelling Ave,None,Snelling,Ave,None,U,05,318,0525850318,T,B,223.445080,"LINESTRING (480782.031 4978437.000, 480688.657..."
329421,Snelling Ave,None,Snelling,Ave,None,U,10,626,1010400626,T,B,103.437352,"LINESTRING (569826.473 5184787.091, 569828.937..."
331330,Snelling Ave,None,Snelling,Ave,None,U,10,626,1010400626,T,B,167.514188,"LINESTRING (569825.626 5184606.498, 569827.562..."
356996,Snelling Ave,None,Snelling,Ave,None,U,10,626,1010400626,T,B,115.285428,"LINESTRING (569828.937 5184890.499, 569830.823..."
403216,Snelling Ave,None,Snelling,Ave,None,U,10,8,1035350008,T,B,131.580222,"LINESTRING (486772.562 4995180.500, 486777.156..."


In [317]:
road_data[(road_data.STREET_NAM == 'MNTH 51')]

IndexError: too many indices for array

Empty GeoDataFrame
Columns: [STREET_NAM, STREET_PRE, BASE_NAME, STREET_TYP, STREET_SUF, E_911, ROUTE_SYST, ROUTE_NUMB, TIS_CODE, DIVIDED_ON, TRAFFIC_DI, SHAPE_Leng, geometry]
Index: []